<div style="display: flex; align-items: flex-start;">
  <div>
      <h1>CS 195: Natural Language Processing</h1>
      <h2>Neural Language Modeling</h2>
      </br>
    <a href="https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F6_1_NeuralLanguageModeling.ipynb">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
  </div>
  <div style="margin-left: 20px;">
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/dalle_neural_net_viz.png?raw=1" width="500" style="display: block;">
  </div>
</div>


**Cover Illustration:** generated by Dall E using the ChatGPT 4 interface, prompted for a visualization of the network used in the code below. *That's not what I meant.*

## Announcement

AI - English Faculty Candidate: Gabriel Ford

Meeting with students: Thursday at 3:30pm in Howard 309

Scholarly Presentation: Friday at 9:00am in Howard ???

## Reference

SLP: Neural Networks and Neural Language Models, Chapter 7 of Speech and Language Processing by Daniel Jurafsky & James H. Martin https://web.stanford.edu/~jurafsky/slp3/7.pdf



In [ ]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 7.6 MB/s eta 0:00:00


## Dataset for today

AG News dataset
* short news articles
* four classes: World, Sports, Business, Sci/Tech

https://huggingface.co/datasets/ag_news


In [ ]:
from datasets import load_dataset
data = load_dataset("ag_news")

print(data["train"]["text"][0])

# 0 is World
# 1 is Sports
# 2 is Business
# 3 is Sci/Tech
print(data["train"]["label"][0])


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


## Review: Text Classification with Keras

Last time, we saw
* how to do text classification when there are more than 2 classes
    - one hot encoded output layer, one node per class, *softmax* output
    - categorical crossentropy loss
* embedding layer
    - pad sequences to all be same length
    - learn vector for each word representing word semantics
    
<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_text_classification.png?raw=1">
</div>

image source: SLP Fig. 7.11, https://web.stanford.edu/~jurafsky/slp3/7.pdf

*pooling* combines/aggregates all of the embeddings in some way

In [ ]:
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, GlobalMaxPooling1D, GlobalAveragePooling1D
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
vocabulary_size = len(tokenizer.word_index) + 1

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])

# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)

#create a neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=max_length))
model.add(Flatten())
#use one of these instead of Flatten() to try a pooling method
#model.add(GlobalMaxPooling1D())
#model.add(GlobalAveragePooling1D())
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/10
3750/3750 [==============================] - 270s 72ms/step - loss: 0.3157 - accuracy: 0.8883
Epoch 2/10
3140/3750 [========================>.....] - ETA: 43s - loss: 0.1081 - accuracy: 0.9641

KeyboardInterrupt: ignored

## Neural Language Modeling

**Neural Language Modeling:** predict next word(s) from previous ones - like what we did with Markov models

Like classification, but output is softmax of every possible word in the vocabulary

Often a first step before extending the model to do summarization, translation, dialog, and other NLP tasks

<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_language_modeling.png?raw=1">
</div>

image source: SLP Fig. 7.13, https://web.stanford.edu/~jurafsky/slp3/7.pdf

## A Neural Language Model in Keras

Let's start by sampling some data from our news dataset

Then split into a training and testing set

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np
import random

data = load_dataset("ag_news")

data_subset, _ = train_test_split(data["train"]["text"],train_size=500)
train_data, test_data = train_test_split(data_subset,train_size=0.8)

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_subset)
vocabulary_size = len(tokenizer.word_index) + 1
print("Vocabulary size:",vocabulary_size)

# Convert text to sequences of integers
train_texts = tokenizer.texts_to_sequences(train_data)




Vocabulary size: 5260


## Preparing training examples

We want to take the sequences like

In [ ]:
print( train_texts[0] )

[1966, 1967, 942, 1968, 272, 121, 1822, 3691, 1966, 77, 47, 69, 20, 1969, 19, 942, 1968, 272, 2, 3692, 239, 7, 233, 3693]


and slide a window across to predict the next word

In [ ]:
print("Use",train_texts[0][0:5],"to predict",train_texts[0][5])
print("Use",train_texts[0][1:6],"to predict",train_texts[0][6])
print("Use",train_texts[0][2:7],"to predict",train_texts[0][7])
print("Use",train_texts[0][3:8],"to predict",train_texts[0][8])
print("etc.")

Use [1966, 1967, 942, 1968, 272] to predict 121
Use [1967, 942, 1968, 272, 121] to predict 1822
Use [942, 1968, 272, 121, 1822] to predict 3691
Use [1968, 272, 121, 1822, 3691] to predict 1966
etc.


## Group Discussion

What data structures (lists, matrixes, etc.) do we need to prepare to make this a classification problem?

Lists can loop and find the right answer

## Preparing all of the examples

In [ ]:
# Decide the sequence length
sequence_length = 5  # Length of the input sequence before predicting the next word

# Create the sequences
predictor_sequences = []
targets = []
for text in train_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences.append(curr_predictor_sequence)
        targets.append(curr_target)


print("Number of sequences:",len(predictor_sequences))


print("First train text:",train_texts[0])
print("Example sequence 0:",predictor_sequences[0]," target:",targets[0])
print("Example sequence 1:",predictor_sequences[1]," target:",targets[1])
print("Example sequence 2:",predictor_sequences[2]," target:",targets[2])
print("Example sequence 3:",predictor_sequences[3]," target:",targets[3])
print("Example sequence 4:",predictor_sequences[4]," target:",targets[4])
print("Example sequence 5:",predictor_sequences[5]," target:",targets[5])


Number of sequences: 13242
First train text: [1966, 1967, 942, 1968, 272, 121, 1822, 3691, 1966, 77, 47, 69, 20, 1969, 19, 942, 1968, 272, 2, 3692, 239, 7, 233, 3693]
Example sequence 0: [1966, 1967, 942, 1968, 272]  target: 121
Example sequence 1: [1967, 942, 1968, 272, 121]  target: 1822
Example sequence 2: [942, 1968, 272, 121, 1822]  target: 3691
Example sequence 3: [1968, 272, 121, 1822, 3691]  target: 1966
Example sequence 4: [272, 121, 1822, 3691, 1966]  target: 77
Example sequence 5: [121, 1822, 3691, 1966, 77]  target: 47


## Padding

Some of the sequences might be really short - so we'll pad them just in case.

In [ ]:
# Pad sequences to ensure uniform length
predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

## The target output

Since we're making this into a classification problem, the output layer needs to have one node for each word in the vocabulary.

Target values need to be transformed into a one-hot encoded vector

In [ ]:

# Convert output to one-hot encoding
target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

print("Predictors words 0:", predictor_sequences_padded[0])
print("target word 0:", targets[0])
print("target word 0 one hot encoded:", target_word_one_hot[0])

Predictors words 0: [1966 1967  942 1968  272]
target word 0: 121
target word 0 one hot encoded: [0. 0. 0. ... 0. 0. 0.]


## Preparing the test set

We have to do all of those same things for the test set.

**Group Exercise:** Turn this into a function so that you can use it to prepare both the training and test sets.

In [ ]:
#test_texts = tokenizer.texts_to_sequences(test_data)

def prepare_examples(seq_length, test_training, tokenizer):
  texts = tokenizer.texts_to_sequences(test_training)
  vocabulary_size = len(tokenizer.word_index) + 1
  # Create the sequences
  predictor_sequences_test = []
  targets_test = []
  for text in texts:
      for i in range(seq_length, len(text)):
          # Take the sequence of tokens as input and the next token as target
          curr_target = text[i]
          curr_predictor_sequence = text[i-seq_length:i]
          predictor_sequences_test.append(curr_predictor_sequence)
          targets_test.append(curr_target)

  # Pad sequences to ensure uniform length
  predictor_sequences_padded_test = pad_sequences(predictor_sequences_test, maxlen=seq_length, padding='pre')

  # Convert target to one-hot encoding
  target_word_one_hot_test = to_categorical(targets_test, num_classes=vocabulary_size)

  return predictor_sequences_padded_test, target_word_one_hot_test


predictor_sequences_padded_test, target_word_one_hot_test = prepare_examples(5, test_data, tokenizer)
predictor_sequences_padded, target_word_one_hot = prepare_examples(5, train_data, tokenizer)


## Designing the Neural Network

We'll start with a simple network like the one we used for text classification.

In [ ]:

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(vocabulary_size, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model - you can also pass in the test set
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, verbose=1, validation_data=(predictor_sequences_padded_test, target_word_one_hot_test))

# The model can now be used to predict the next word in a sequence

Epoch 1/10
414/414 [==============================] - 12s 27ms/step - loss: 7.7065 - accuracy: 0.0463 - val_loss: 7.4046 - val_accuracy: 0.0524
Epoch 2/10
414/414 [==============================] - 10s 24ms/step - loss: 6.8834 - accuracy: 0.0477 - val_loss: 7.6180 - val_accuracy: 0.0568
Epoch 3/10
414/414 [==============================] - 9s 22ms/step - loss: 6.5073 - accuracy: 0.0575 - val_loss: 7.9362 - val_accuracy: 0.0724
Epoch 4/10
414/414 [==============================] - 11s 27ms/step - loss: 6.2085 - accuracy: 0.0732 - val_loss: 8.3151 - val_accuracy: 0.0791
Epoch 5/10
414/414 [==============================] - 10s 24ms/step - loss: 5.9087 - accuracy: 0.0921 - val_loss: 8.6667 - val_accuracy: 0.0830
Epoch 6/10
414/414 [==============================] - 8s 19ms/step - loss: 5.5551 - accuracy: 0.1211 - val_loss: 9.1829 - val_accuracy: 0.0785
Epoch 7/10
414/414 [==============================] - 9s 22ms/step - loss: 5.1379 - accuracy: 0.1593 - val_loss: 9.7423 - val_accuracy: 0.

## Testing the final model on the test set

In [ ]:
loss, accuracy = model.evaluate(predictor_sequences_padded_test, target_word_one_hot_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

113/113 [==============================] - 1s 6ms/step - loss: 13.0385 - accuracy: 0.0638
Test accuracy: 6.38%


## Text Generation

We can use this model to successively generate words based on previous ones - like our Markov sequence on steroids.

Let's see how this works

In [ ]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
print(tokens_list)

tokens_array = np.array(tokens_list)
print(tokens_array)

[[1, 164, 18, 13, 22]]
[[  1 164  18  13  22]]


the model will predict probabilities for each possible word in the output

In [ ]:
predicted_probabilities = model.predict(tokens_array,verbose=0)
print(predicted_probabilities)
print("We get a probability for each of the",len(predicted_probabilities[0]),"words")

[[6.4748105e-15 1.9881995e-04 5.3993557e-03 ... 2.3984421e-09
  1.0786780e-08 1.7485203e-07]]
We get a probability for each of the 5211 words


then we could get the word associated with the highest probability

In [ ]:
predicted_index = np.argmax(predicted_probabilities)
print("word index:",predicted_index)
predicted_word = tokenizer.index_word[predicted_index]
print("word:",predicted_word)

word index: 20
word: is


or you could generate a random word according the these probabilities (like with did with Markov text generation)

### putting it all together

In [ ]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
tokens = tokens_list[0]

for i in range(50):
    curr_seq = tokens[-sequence_length:]
    curr_array = np.array([curr_seq])
    predicted_probabilities = model.predict(curr_array,verbose=0)
    predicted_index = np.argmax(predicted_probabilities)
    predicted_word = tokenizer.index_word[predicted_index]
    print(predicted_word+" ",end="")
    tokens.append(predicted_index)


is the company is not be their week to beat 160 on thursday but more than four years ago the british 39 s largest camera volleyball is is that the two 39 s most tennis bronze medal at one of its killed and two million in a new quarter news 

## Applied Exploration

Pick another dataset and get it working with this code
* you will likely need to prepare the text a little differently - do you need to first break it into sentences?
* describe your dataset and what you did to prepare it

Perform a neural language modeling experiment
* experiment with something to try to find a well-performing model
    * sliding window size
    * number of hidden nodes in the network
    * learning rate
* describe what you did and write up an interpretation of your results

#APPLIED EXPLORATION NOTES:


DATASET PREP:
  - The dataset I used was a yelp review dataset that assigned numbers based on the stars of the review.
  - I didn't need to do much dataset preparation because the text data was the same format as the text we used in the other model.

MODELS:
  - I created a basic model with the previous parameters from the original model and ran it.
  - I then created a tuned model, where I added another hidden layer with a elu activation function. I also added a optimizer where I adjusted the learning rate.

RESULTS:

  - Overall, I couldn't get the tuned model to perform better than the original model. The overall accuracy was a little better but the validation accuracy was not great.

In [ ]:
from keras import optimizers

#an example on changing the learning rate
optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=["accuracy"])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np
import random

data = load_dataset("yelp_review_full")

data_subset, _ = train_test_split(data["train"]["text"],train_size=500)
train_data, test_data = train_test_split(data_subset,train_size=0.8)

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_subset)
vocabulary_size = len(tokenizer.word_index) + 1
print("Vocabulary size:",vocabulary_size)

# Convert text to sequences of integers
train_texts = tokenizer.texts_to_sequences(train_data)



Vocabulary size: 7399


In [ ]:
# Decide the sequence length
sequence_length = 5  # Length of the input sequence before predicting the next word

# Create the sequences
predictor_sequences = []
targets = []
for text in train_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences.append(curr_predictor_sequence)
        targets.append(curr_target)


print("Number of sequences:",len(predictor_sequences))


print("First train text:",train_texts[0])
print("Example sequence 0:",predictor_sequences[0]," target:",targets[0])
print("Example sequence 1:",predictor_sequences[1]," target:",targets[1])
print("Example sequence 2:",predictor_sequences[2]," target:",targets[2])
print("Example sequence 3:",predictor_sequences[3]," target:",targets[3])
print("Example sequence 4:",predictor_sequences[4]," target:",targets[4])
print("Example sequence 5:",predictor_sequences[5]," target:",targets[5])


Number of sequences: 51404
First train text: [341, 609, 7, 248, 2578, 26, 181, 16, 5409, 91, 938, 1516, 762, 2038, 893, 448, 95, 120, 35, 360, 5, 158, 10, 149]
Example sequence 0: [341, 609, 7, 248, 2578]  target: 26
Example sequence 1: [609, 7, 248, 2578, 26]  target: 181
Example sequence 2: [7, 248, 2578, 26, 181]  target: 16
Example sequence 3: [248, 2578, 26, 181, 16]  target: 5409
Example sequence 4: [2578, 26, 181, 16, 5409]  target: 91
Example sequence 5: [26, 181, 16, 5409, 91]  target: 938


In [ ]:
# Pad sequences to ensure uniform length
predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

In [ ]:
# Convert output to one-hot encoding
target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

print("Predictors words 0:", predictor_sequences_padded[0])
print("target word 0:", targets[0])
print("target word 0 one hot encoded:", target_word_one_hot[0])

Predictors words 0: [ 341  609    7  248 2578]
target word 0: 26
target word 0 one hot encoded: [0. 0. 0. ... 0. 0. 0.]


In [ ]:
#test_texts = tokenizer.texts_to_sequences(test_data)

def prepare_examples(seq_length, test_training, tokenizer):
  texts = tokenizer.texts_to_sequences(test_training)
  vocabulary_size = len(tokenizer.word_index) + 1
  # Create the sequences
  predictor_sequences_test = []
  targets_test = []
  for text in texts:
      for i in range(seq_length, len(text)):
          # Take the sequence of tokens as input and the next token as target
          curr_target = text[i]
          curr_predictor_sequence = text[i-seq_length:i]
          predictor_sequences_test.append(curr_predictor_sequence)
          targets_test.append(curr_target)

  # Pad sequences to ensure uniform length
  predictor_sequences_padded_test = pad_sequences(predictor_sequences_test, maxlen=seq_length, padding='pre')

  # Convert target to one-hot encoding
  target_word_one_hot_test = to_categorical(targets_test, num_classes=vocabulary_size)

  return predictor_sequences_padded_test, target_word_one_hot_test


predictor_sequences_padded_test, target_word_one_hot_test = prepare_examples(5, test_data, tokenizer)
predictor_sequences_padded, target_word_one_hot = prepare_examples(5, train_data, tokenizer)


##BASE MODEL

In [ ]:

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(vocabulary_size, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model - you can also pass in the test set
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, verbose=1, validation_data=(predictor_sequences_padded_test, target_word_one_hot_test))

# The model can now be used to predict the next word in a sequence

Epoch 1/10
1668/1668 [==============================] - 50s 29ms/step - loss: 6.8776 - accuracy: 0.0531 - val_loss: 6.6923 - val_accuracy: 0.0668
Epoch 2/10
1668/1668 [==============================] - 51s 31ms/step - loss: 6.1639 - accuracy: 0.0835 - val_loss: 6.5641 - val_accuracy: 0.0921
Epoch 3/10
1668/1668 [==============================] - 50s 30ms/step - loss: 5.7103 - accuracy: 0.1090 - val_loss: 6.6573 - val_accuracy: 0.1042
Epoch 4/10
1668/1668 [==============================] - 49s 29ms/step - loss: 5.3363 - accuracy: 0.1324 - val_loss: 6.8450 - val_accuracy: 0.1053
Epoch 5/10
1668/1668 [==============================] - 48s 28ms/step - loss: 4.9682 - accuracy: 0.1592 - val_loss: 7.1082 - val_accuracy: 0.1030
Epoch 6/10
1668/1668 [==============================] - 49s 30ms/step - loss: 4.5928 - accuracy: 0.1905 - val_loss: 7.4688 - val_accuracy: 0.1023
Epoch 7/10
1668/1668 [==============================] - 51s 31ms/step - loss: 4.2049 - accuracy: 0.2254 - val_loss: 7.9796 -

In [ ]:
loss, accuracy = model.evaluate(predictor_sequences_padded_test, target_word_one_hot_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

497/497 [==============================] - 4s 9ms/step - loss: 10.4511 - accuracy: 0.0786
Test accuracy: 7.86%


##IMPROVED MODEL

In [ ]:

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="elu"))
model.add(Dense(vocabulary_size, activation='softmax'))

# Compile the model
from keras import optimizers

#an example on changing the learning rate
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=["accuracy"])

# Fit the model - you can also pass in the test set
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, verbose=1, validation_data=(predictor_sequences_padded_test, target_word_one_hot_test))

# The model can now be used to predict the next word in a sequence

Epoch 1/10
1607/1607 [==============================] - 113s 70ms/step - loss: 6.8704 - accuracy: 0.0502 - val_loss: 6.7915 - val_accuracy: 0.0609
Epoch 2/10
1607/1607 [==============================] - 34s 21ms/step - loss: 6.1827 - accuracy: 0.0781 - val_loss: 6.5322 - val_accuracy: 0.0950
Epoch 3/10
1607/1607 [==============================] - 40s 25ms/step - loss: 5.6766 - accuracy: 0.1116 - val_loss: 6.6984 - val_accuracy: 0.0980
Epoch 4/10
1607/1607 [==============================] - 37s 23ms/step - loss: 5.2548 - accuracy: 0.1354 - val_loss: 6.9457 - val_accuracy: 0.0952
Epoch 5/10
1607/1607 [==============================] - 35s 21ms/step - loss: 4.8358 - accuracy: 0.1657 - val_loss: 7.3972 - val_accuracy: 0.0929
Epoch 6/10
1607/1607 [==============================] - 37s 23ms/step - loss: 4.4171 - accuracy: 0.2022 - val_loss: 7.8985 - val_accuracy: 0.0876
Epoch 7/10
1607/1607 [==============================] - 37s 23ms/step - loss: 3.9957 - accuracy: 0.2471 - val_loss: 8.4551 

In [ ]:
loss, accuracy = model.evaluate(predictor_sequences_padded_test, target_word_one_hot_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

428/428 [==============================] - 5s 11ms/step - loss: 10.7151 - accuracy: 0.0661
Test accuracy: 6.61%
